In [ ]:
!pip install --upgrade imutils

In [ ]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/facial-landmarks/facial-landmarks.zip
!unzip -qq facial-landmarks.zip
%cd facial-landmarks

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/capstone project")

In [ ]:
from keras.layers import Input, Conv2D, Lambda, merge, Dense, Flatten,MaxPooling2D,Activation, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from skimage.io import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import numpy as np
from keras.preprocessing import image
import tensorflow as tf

In [ ]:
nameDir = [name for name in os.listdir('Mouth_model/proc_images/')]
folderlist = []
for fname in nameDir:
    path='/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/proc_images/'+fname+'/'
    folderlist.append(path)

In [ ]:
folderlist[2]

'/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/proc_images/Alia Bhatt/'

In [ ]:
left_input_name = []
right_input_name = []
targets = []

In [ ]:
# for i in range(0, 29):
#   fnamei = [folderlist[i] + name for name in os.listdir(folderlist[i])]
#   for j in range(1,10):
#     for k in range(j):
#       left_input_name.append(fnamei[j])
#       right_input_name.append(fnamei[k])
#       targets.append(1)

In [ ]:
for i in range(0, 27):
  fnamei = [folderlist[i] + name for name in os.listdir(folderlist[i])]
  tempSet = set([])
  while len(tempSet) < 66:
    y1 = random.randint(0,len(fnamei)-1)
    y2 = random.randint(0,len(fnamei)-1)
    if y1 == y2:
      continue
    myTuple = (y1, y2)
    if myTuple in tempSet:
      continue
    myTuple2 = (y2, y1)
    if myTuple2 in tempSet:
      continue
    tempSet.add(myTuple)
    left_input_name.append(fnamei[y1])
    right_input_name.append(fnamei[y2])
    targets.append(1)

In [ ]:
len(targets)

1782

In [ ]:
mySet = set([])
while len(mySet) < 1782:
  x1 = random.randint(0,26)
  x2 = random.randint(0,26)
  if x1 == x2:
    continue
  fname1 = [folderlist[x1] + name for name in os.listdir(folderlist[x1])]
  fname2 = [folderlist[x2] + name for name in os.listdir(folderlist[x2])]
  y1 = random.randint(0,len(fname1)-1)
  y2 = random.randint(0,len(fname2)-1)
  myTuple = (x1,y1,x2,y2)
  if myTuple in mySet:
    continue
  myTuple2 = (x2,y2,x1,y1)
  if myTuple2 in mySet:
    continue
  mySet.add(myTuple)
  left_input_name.append(fname1[y1])
  right_input_name.append(fname2[y2])
  targets.append(0)

In [ ]:
len(mySet)

1782

In [ ]:
len(targets)

3564

In [ ]:
targets_array = np.squeeze(np.array(targets))
targets_array

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
targets_array = targets_array.astype('float64')
targets_array

array([1., 1., 1., ..., 0., 0., 0.])

In [ ]:
left_input_data = np.array([image.img_to_array(image.load_img(fname, target_size=(32, 80))) for fname in left_input_name])
left_input_data = np.squeeze(left_input_data)
left_input_data.shape

(3564, 32, 80, 3)

In [ ]:
right_input_data = np.array([image.img_to_array(image.load_img(fname, target_size=(32, 80))) for fname in right_input_name])
right_input_data = np.squeeze(right_input_data)
right_input_data.shape

(3564, 32, 80, 3)

In [ ]:
BASE_PATH = "/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model"

In [ ]:
file_name = "left_input_data"
np.save(os.path.join(BASE_PATH, file_name), left_input_data)

In [ ]:
file_name = "right_input_data"
np.save(os.path.join(BASE_PATH, file_name), right_input_data)

In [ ]:
file_name = "targets_array"
np.save(os.path.join(BASE_PATH, file_name), targets_array)

In [ ]:
left_input_data = np.load('/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/left_input_data.npy')
right_input_data = np.load('/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/right_input_data.npy')
targets_array = np.load('/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/targets_array.npy')

In [ ]:
left_input_data = left_input_data/255
right_input_data = right_input_data/255

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
lid_train, lid_test, rid_train, rid_test, y_train, y_test = train_test_split(left_input_data, right_input_data, targets_array, test_size=0.10, random_state=42)

In [ ]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [ ]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [ ]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (5, 5), activation='relu', input_shape=input_shape, kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu',  kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu',  kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='sigmoid',kernel_regularizer=l2(1e-3),kernel_initializer=initialize_weights,bias_initializer=initialize_bias)
    ])
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, (7, 7), activation='relu', input_shape=input_shape, kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      Dropout(0.2),
      tf.keras.layers.Conv2D(128, (5, 5), activation='relu',  kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      Dropout(0.2),
      tf.keras.layers.Conv2D(256, (3, 3), activation='relu',  kernel_initializer=initialize_weights, bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)),
      tf.keras.layers.MaxPooling2D(2, 2),
      Dropout(0.2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(2048, activation='sigmoid',kernel_regularizer=l2(1e-3),kernel_initializer=initialize_weights,bias_initializer=initialize_bias)
    ])
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
model = get_siamese_model((32, 80, 3))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 80, 3)]  0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 32, 80, 3)]  0           []                               
                                                                                                  
 sequential (Sequential)        (None, 2048)         4181632     ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 lambda (Lambda)                (None, 2048)         0           ['sequential[0][0]',         

In [ ]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit([lid_train,rid_train], y_train,
          batch_size=16,
          validation_split=0.1,
          epochs=50,
          verbose=1)

Epoch 1/50
181/181 [==============================] - 15s 20ms/step - loss: 1.0419 - val_loss: 1.0442
Epoch 2/50
181/181 [==============================] - 3s 15ms/step - loss: 0.9824 - val_loss: 1.0032
Epoch 3/50
181/181 [==============================] - 2s 13ms/step - loss: 0.9479 - val_loss: 0.9726
Epoch 4/50
181/181 [==============================] - 2s 11ms/step - loss: 0.9245 - val_loss: 0.9472
Epoch 5/50
181/181 [==============================] - 2s 11ms/step - loss: 0.9004 - val_loss: 0.9259
Epoch 6/50
181/181 [==============================] - 2s 11ms/step - loss: 0.8804 - val_loss: 0.9073
Epoch 7/50
181/181 [==============================] - 2s 11ms/step - loss: 0.8644 - val_loss: 0.8908
Epoch 8/50
181/181 [==============================] - 2s 11ms/step - loss: 0.8499 - val_loss: 0.8748
Epoch 9/50
181/181 [==============================] - 2s 11ms/step - loss: 0.8388 - val_loss: 0.8452
Epoch 10/50
181/181 [==============================] - 2s 11ms/step - loss: 0.8275 - val_l

In [ ]:
# save model to .h5 file
model.save("/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/siamese_network.h5")

In [ ]:
#Load model back from saved file
model = get_siamese_model((32, 80, 3))
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)
model.load_weights('/content/drive/My Drive/Colab Notebooks/capstone project/Mouth_model/siamese_network.h5')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
prediction = model.predict([lid_test, rid_test])

In [ ]:
prediction = (model.predict([lid_test, rid_test]) > 0.5).astype("int32")

In [ ]:
prediction.shape

(357, 1)

In [ ]:
y_test.shape

(357,)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         0.0       0.62      0.61      0.61       156
         1.0       0.70      0.71      0.70       201

    accuracy                           0.66       357
   macro avg       0.66      0.66      0.66       357
weighted avg       0.66      0.66      0.66       357



In [ ]:
path="/content/drive/My Drive/Colab Notebooks/Face_rec/bound_photos/Cameron_Diaz/Cameron_Diaz_0004.jpg"
img=image.load_img(path, target_size=(100, 100)) # edit the target_size

x1=image.img_to_array(img)
x1=np.expand_dims(x1, axis=0)

path="/content/drive/My Drive/Colab Notebooks/Face_rec/bound_photos/Nicole_Kidman/Nicole_Kidman_0001.jpg"
img=image.load_img(path, target_size=(100, 100)) # edit the target_size

x2=image.img_to_array(img)
x2=np.expand_dims(x2, axis=0)

predTest = (model.predict([x1, x2]) > 0.5).astype("int32")